In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import defaultdict, Counter
from nltk.probability import FreqDist
import numpy as np
import re
import string
import nltk
import math

In [ ]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Load the data
df = pd.read_csv('/content/df_balanced_unshuffled_4columns.csv')

In [ ]:
# Concatenate the list of sentences into a single sentence
df['text'] = df['text'].apply(lambda x: ' '.join(eval(x)))

In [ ]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing brackets and special characters
    text = re.sub(r'\[.*?\]', '', text)
    # Removing specific special characters: < > \ / , '
    text = re.sub(r'[<>\\/,\'"]', '', text)
    # Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenizing
    tokens = word_tokenize(text)
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [ ]:
# Apply preprocessing
df['text'] = df['text'].apply(preprocess_text)

In [ ]:
############################################################
############################################################
#PERPLEXITY#
############################################################
############################################################

In [ ]:
perplexity_df = df.copy()

In [ ]:
# Tokenize the concatenated text into sentences
perplexity_df['sentences'] = perplexity_df['text'].apply(sent_tokenize)

In [ ]:
# Flatten the sentences into a list of lists of tokens
all_sentences = [word_tokenize(sentence) for sentences in perplexity_df['sentences'] for sentence in sentences]

In [ ]:
# Create bigrams (2-grams)
bigrams = [(w1, w2) for sentence in all_sentences for w1, w2 in zip(sentence[:-1], sentence[1:])]

# Count bigram and unigram frequencies
bigram_freq = Counter(bigrams)
unigram_freq = Counter([word for sentence in all_sentences for word in sentence])

# Function to calculate perplexity
def calculate_perplexity(sentence):
    tokens = word_tokenize(sentence)
    bigrams = [(tokens[i], tokens[i+1]) for i in range(len(tokens)-1)]
    vocab_size = len(unigram_freq)

    probs = []
    for w1, w2 in bigrams:
        prob = (bigram_freq[(w1, w2)] + 1) / (unigram_freq[w1] + vocab_size)  # Add-1 smoothing
        probs.append(prob)

    if probs:
        perplexity = np.exp(-np.mean(np.log(probs)))
        return perplexity
    else:
        return np.nan


# Tokenize the sentences and calculate perplexities
def tokenize_sentences(text):
    sentences = sent_tokenize(text)
    return pd.Series(sentences, index=[f'sentence{i+1}' for i in range(len(sentences))])

tokenized_df = perplexity_df['text'].apply(tokenize_sentences)
final_df_perplexity = pd.concat([tokenized_df, perplexity_df[['label']]], axis=1)


# Apply perplexity calculation
def apply_perplexity(row):
    sentence_cols = [col for col in row.index if col.startswith('sentence')]
    perplexities = [calculate_perplexity(row[col]) for col in sentence_cols if not pd.isna(row[col])]
    return pd.Series({
        'mean_perplexity': np.mean(perplexities),
        'median_perplexity': np.median(perplexities),
        'std_perplexity': np.std(perplexities)
    })

# Apply perplexity calculation row-wise
perplexity_results = final_df_perplexity.apply(apply_perplexity, axis=1)

# Convert perplexity_results to DataFrame
perplexity_results_df = pd.DataFrame(perplexity_results, columns=['mean_perplexity', 'median_perplexity', 'std_perplexity'])

# Combine perplexity results with original dataframe
final_df_perplexity = pd.concat([perplexity_df['text'], perplexity_results_df], axis=1)


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
final_df_perplexity.head(500)

,text,mean_perplexity,median_perplexity,std_perplexity
0,windows aligned structural spars aircraft fuse...,33594.317670,33594.317670,0.0
1,tried bacon really fucking good widely acknowl...,28637.472335,28637.472335,0.0
2,opinion common nowadays two three decades ago ...,47428.782080,47428.782080,0.0
3,walk cop tell take ecstasy every day prove thi...,18049.460322,18049.460322,0.0
4,compressor ac nt burn fuel driven car engine v...,20317.841977,20317.841977,0.0
...,...,...,...,...
495,closest thing looking would forex exchanges cu...,31130.108295,31130.108295,0.0
496,bit switch zero 1 byte 8 bits byte would look ...,17278.289371,17278.289371,0.0
497,community policing mostly staff check articles...,32802.851508,32802.851508,0.0
498,real simple definition analogy present value w...,23489.076846,23489.076846,0.0


In [ ]:
# Save the DataFrame to a CSV file
output_path = '/content/final_df_perplexity.csv'
final_df_perplexity.to_csv(output_path, index=False)

In [ ]:
############################################################
############################################################
#BURSTINESS#
############################################################
############################################################

In [ ]:
burstiness_df = df.copy()

In [ ]:
burstiness_df = burstiness_df[['text', 'label']]
burstiness_df.head(2)

,text,label
0,windows aligned structural spars aircraft fuse...,0
1,tried bacon really fucking good widely acknowl...,0


In [ ]:
burstiness_df.shape

(47734, 2)

In [ ]:
def compute_burstiness(text):
    words = preprocess_text(text)
    #words = text.split()
    word_counts={}
    for word in words:
      if word in word_counts:
        word_counts[word]+=1
      else:
        word_counts[word]=1
    frequencies = np.array(list(word_counts.values()))
    mean_freq = np.mean(frequencies)
    variance = np.var(frequencies)
    std_dev = variance**0.5;
    # burstiness_score = (std_dev - mean_freq) / (std_dev + mean_freq)
    burstiness_score = (variance - mean_freq) / (variance + mean_freq)
    # burstiness_score = variance/mean_freq**2
    return burstiness_score

burstiness_df['burstiness1'] = burstiness_df['text'].apply(compute_burstiness)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [ ]:
burstiness_df.head(2);

In [ ]:
burstiness_df.head(2);

In [ ]:
def compute_burstiness(text):
    words = text.split()
    word_counts={}
    for word in words:
      if word in word_counts:
        word_counts[word]+=1
      else:
        word_counts[word]=1
    frequencies = np.array(list(word_counts.values()))
    mean_freq = np.mean(frequencies)
    variance = np.var(frequencies)
    burstiness_score = variance/mean_freq**2
    return burstiness_score

burstiness_df['burstiness2'] = burstiness_df['text'].apply(compute_burstiness)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [ ]:
burstiness_df.head()

,text,label,burstiness1,burstiness2
0,windows aligned structural spars aircraft fuse...,0,0.871553,0.261974
1,tried bacon really fucking good widely acknowl...,0,0.578668,0.172023
2,opinion common nowadays two three decades ago ...,0,0.874788,0.162393
3,walk cop tell take ecstasy every day prove thi...,0,0.886011,0.118802
4,compressor ac nt burn fuel driven car engine v...,0,0.939385,0.917867


In [ ]:
# Save the DataFrame to a CSV file
output_path = '/content/final_df_burstiness.csv'
burstiness_df.to_csv(output_path, index=False)

In [ ]:
############################################################
############################################################
#READABILITY SCORE#
############################################################
############################################################

In [ ]:
readabilityScore_df = df.copy()

In [ ]:
readabilityScore_df.head()

,Unnamed: 0,text,avg line length,word density,label
0,0,windows aligned structural spars aircraft fuse...,26.333333,66.455696,0
1,1,tried bacon really fucking good widely acknowl...,13.800000,68.115942,0
2,2,opinion common nowadays two three decades ago ...,25.222222,55.947137,0
3,3,walk cop tell take ecstasy every day prove thi...,22.454545,56.680162,0
4,4,compressor ac nt burn fuel driven car engine v...,23.000000,44.927536,0


In [ ]:
readabilityScore_df = readabilityScore_df[['text', 'label']]
readabilityScore_df.head(2)

,text,label
0,windows aligned structural spars aircraft fuse...,0
1,tried bacon really fucking good widely acknowl...,0


In [ ]:
# Function to count syllables
def count_syllables(word):
    return max(1, len(re.findall(r'[aeiou]', word, re.I)))

# Function to calculate Flesch-Kincaid Grade Level
def flesch_kincaid_grade(text):
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    num_words = len(words)
    num_sentences = len(sentences)
    num_syllables = sum(count_syllables(word) for word in words)

    if num_sentences == 0 or num_words == 0:
        return 0

    return 0.39 * (num_words / num_sentences) + 11.8 * (num_syllables / num_words) - 15.59

# Function to calculate Gunning Fog Index
def gunning_fog(text):
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    num_words = len(words)
    num_sentences = len(sentences)
    complex_words = len([word for word in words if count_syllables(word) > 2])

    if num_sentences == 0 or num_words == 0:
        return 0

    return 0.4 * ((num_words / num_sentences) + 100 * (complex_words / num_words))

# Function to calculate SMOG Index
def smog_index(text):
    sentences = sent_tokenize(text)
    num_sentences = len(sentences)
    polysyllable_count = len([word for word in word_tokenize(text) if count_syllables(word) > 2])

    if num_sentences < 30:
        return 0

    return 1.0430 * math.sqrt(polysyllable_count * (30 / num_sentences)) + 3.1291

# Function to calculate readability scores
def calculate_readability_scores(text):


    fk = flesch_kincaid_grade(text)
    gf = gunning_fog(text)
    smog = smog_index(text)

    return fk, gf, smog

# Apply the function to the 'text' column and create new columns
readabilityScore_df['flesch_kincaid_score'], readabilityScore_df['gunning_fog_score'], readabilityScore_df['smog_index_score'] = zip(*readabilityScore_df['text'].apply(calculate_readability_scores))



In [ ]:
readabilityScore_df.head()

,text,label,flesch_kincaid_score,gunning_fog_score,smog_index_score
0,windows aligned structural spars aircraft fuse...,0,39.775821,42.322388,0
1,tried bacon really fucking good widely acknowl...,0,21.597391,23.113043,0
2,opinion common nowadays two three decades ago ...,0,45.540256,49.661538,0
3,walk cop tell take ecstasy every day prove thi...,0,43.805000,47.927273,0
4,compressor ac nt burn fuel driven car engine v...,0,67.532667,70.666667,0


In [ ]:
# Save the DataFrame to a CSV file
output_path = '/content/final_df_readabilityScore.csv'
readabilityScore_df.to_csv(output_path, index=False)

In [ ]:
############################################################
############################################################
#DATASET PREPARATION#
############################################################
############################################################

In [ ]:
# Save the DataFrame to a CSV file
output_path = '/content/df.csv'
df.to_csv(output_path, index=False)

In [ ]:
df1= pd.read_csv("/content/df.csv")
df2= pd.read_csv("/content/final_df_perplexity.csv")
df3= pd.read_csv("/content/final_df_burstiness.csv")
df4= pd.read_csv("/content/final_df_readabilityScore.csv")

In [ ]:
df1.head()

,Unnamed: 0,text,avg line length,word density,label
0,0,windows aligned structural spars aircraft fuse...,26.333333,66.455696,0
1,1,tried bacon really fucking good widely acknowl...,13.800000,68.115942,0
2,2,opinion common nowadays two three decades ago ...,25.222222,55.947137,0
3,3,walk cop tell take ecstasy every day prove thi...,22.454545,56.680162,0
4,4,compressor ac nt burn fuel driven car engine v...,23.000000,44.927536,0


In [ ]:
df2.head()

,text,mean_perplexity,median_perplexity,std_perplexity
0,windows aligned structural spars aircraft fuse...,33594.317670,33594.317670,0.0
1,tried bacon really fucking good widely acknowl...,28637.472335,28637.472335,0.0
2,opinion common nowadays two three decades ago ...,47428.782080,47428.782080,0.0
3,walk cop tell take ecstasy every day prove thi...,18049.460322,18049.460322,0.0
4,compressor ac nt burn fuel driven car engine v...,20317.841977,20317.841977,0.0


In [ ]:
df3.head()

,text,label,burstiness1,burstiness2
0,windows aligned structural spars aircraft fuse...,0,0.871553,0.261974
1,tried bacon really fucking good widely acknowl...,0,0.578668,0.172023
2,opinion common nowadays two three decades ago ...,0,0.874788,0.162393
3,walk cop tell take ecstasy every day prove thi...,0,0.886011,0.118802
4,compressor ac nt burn fuel driven car engine v...,0,0.939385,0.917867


In [ ]:
df4.head()

,text,label,flesch_kincaid_score,gunning_fog_score,smog_index_score
0,windows aligned structural spars aircraft fuse...,0,39.775821,42.322388,0
1,tried bacon really fucking good widely acknowl...,0,21.597391,23.113043,0
2,opinion common nowadays two three decades ago ...,0,45.540256,49.661538,0
3,walk cop tell take ecstasy every day prove thi...,0,43.805000,47.927273,0
4,compressor ac nt burn fuel driven car engine v...,0,67.532667,70.666667,0


In [ ]:
# Select desired columns
columns_from_df1 = df1[['text', 'avg line length', 'word density']]
columns_from_df2 = df2[['mean_perplexity']]
columns_from_df3 = df3[['burstiness1', 'burstiness2']]
columns_from_df4 = df4[['flesch_kincaid_score', 'gunning_fog_score']]
label_column = df1[['label']]

# Concatenate selected columns
correct_model_df = pd.concat([columns_from_df1, columns_from_df2, columns_from_df3, columns_from_df4, label_column], axis=1)

# Save the DataFrame to a CSV file
output_path = '/content/correct_model_df.csv'
correct_model_df.to_csv(output_path, index=False)


In [ ]:
correct_model_df.head()

,text,avg line length,word density,mean_perplexity,burstiness1,burstiness2,flesch_kincaid_score,gunning_fog_score,label
0,windows aligned structural spars aircraft fuse...,26.333333,66.455696,33594.317670,0.871553,0.261974,39.775821,42.322388,0
1,tried bacon really fucking good widely acknowl...,13.800000,68.115942,28637.472335,0.578668,0.172023,21.597391,23.113043,0
2,opinion common nowadays two three decades ago ...,25.222222,55.947137,47428.782080,0.874788,0.162393,45.540256,49.661538,0
3,walk cop tell take ecstasy every day prove thi...,22.454545,56.680162,18049.460322,0.886011,0.118802,43.805000,47.927273,0
4,compressor ac nt burn fuel driven car engine v...,23.000000,44.927536,20317.841977,0.939385,0.917867,67.532667,70.666667,0


In [ ]:
from google.colab import files
final_df_perplexity.to_csv('final_df_perplexity.csv', index=False)

# Download the CSV file
files.download('final_df_perplexity.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
burstiness_df.to_csv('final_df_burstiness.csv', index=False)

# Download the CSV file
files.download('final_df_burstiness.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
readabilityScore_df.to_csv('final_df_readabilityScore.csv', index=False)

# Download the CSV file
files.download('final_df_readabilityScore.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>